# Project: Phone Selection with Data Envelopment Analysis

Data obtained from the following resource (code was not included): https://vc.bridgew.edu/cgi/viewcontent.cgi?article=1355&context=undergrad_rev



In [9]:
# Load the packages
#!pip install pulp 
import pulp
import numpy as np
import pandas as pd

# load the dataset
df = pd.read_csv('data_phone_selection.csv')

# Define inputs and outputs
inputs = df[['baterry_mah', 'weight_oz', 'camera_mp', 'standby_time_hours']].values
outputs = df['retail_price_usd'].values

# Calculate efficiencies of each DMU using DEA with linear programming
efficiencies = []
num_inputs = inputs.shape[1]


for di in range(len(inputs)):
    prob = pulp.LpProblem("DEA", pulp.LpMaximize)
    w_in = pulp.LpVariable.dicts("w_in", range(num_inputs), lowBound=0)
    w_out = pulp.LpVariable("w_out", lowBound=0)
    
    prob += w_out * outputs[di] - sum(w_in[i] * inputs[di][i] for i in range(num_inputs))
    
    # Add constraints to ensure no DMU's efficiency exceeds 1
    for i in range(len(inputs)):
        prob += w_out * outputs[i] - sum(w_in[j] * inputs[i][j] for j in range(num_inputs)) <= 1
    
    prob.solve()
    
    if prob.status == pulp.LpStatusOptimal:
        efficiencies.append(min(pulp.value(prob.objective), 1))
    else:
        efficiencies.append(None)

# Display efficiencies
for i, eff in enumerate(efficiencies):
    print(f"Efficiency of DMU {i + 1}: {eff}")

# Optional: Add efficiencies to the dataframe and save to a new CSV
df['efficiency'] = efficiencies
#df.to_csv('data_phone_selection_with_efficiencies.csv', index=False)

Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Dec 15 2019 

command line - /Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/pulp/solverdir/cbc/osx/64/cbc /var/folders/8j/3pr9zqy97g998nn_l0_dmlz80000gn/T/c95c9eb2ef514ba5946b9f342c13361c-pulp.mps -max -timeMode elapsed -branch -printingOptions all -solution /var/folders/8j/3pr9zqy97g998nn_l0_dmlz80000gn/T/c95c9eb2ef514ba5946b9f342c13361c-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 13 COLUMNS
At line 59 RHS
At line 68 BOUNDS
At line 69 ENDATA
Problem MODEL has 8 rows, 5 columns and 40 elements
Coin0008I MODEL read with 0 errors
Option for timeMode changed from cpu to elapsed
Presolve 8 (0) rows, 5 (0) columns and 40 (0) elements
0  Obj -0 Dual inf 1476.0082 (1)
0  Obj -0 Dual inf 1476.0082 (1)
3  Obj 1
Optimal - objective value 1
Optimal objective 1 - 3 iterations time 0.002
Option for printingOptions changed from normal to all
Total time (CPU seconds):  

In [10]:
df

,decision_making_unit,baterry_mah,weight_oz,camera_mp,standby_time_hours,retail_price_usd,efficiency
0,iPhone 5,1800,3.95,8.0,225.0,359.99,1.000000
1,HTC Windows 8,1800,4.60,8.0,300.0,399.99,1.000000
2,HTC Desire,1230,5.15,5.0,384.0,299.99,1.000000
3,Samsung Galaxy S4,2600,4.59,13.0,370.0,499.99,1.000000
4,Samsung Galaxy S4 Mini,1900,3.77,8.0,300.0,399.99,1.000000
5,Samsung Gusto 3,1000,3.50,1.3,770.4,149.99,1.000000
6,LG Cosmos,950,4.58,1.3,818.0,149.99,1.000000
7,Nokia Lumia 928,2000,5.75,8.7,606.0,399.99,0.941664


Convert into function to customize the data, inputs and outputs variables, and option to download



In [16]:
# Function to calculate the efficiencies using Data Envelopment Analysis
# works for multiple inputs and outputs to make the calculations
import pandas as pd
import numpy as np
import pulp

def calculate_efficiencies(data_path, input_features, output_features, output_path=None):
    # Load the dataset
    df = pd.read_csv(data_path)
    
    # Define inputs and outputs
    inputs = df[input_features].values
    outputs = df[output_features].values
    
    # Calculate efficiencies of each DMU using DEA with linear programming
    efficiencies = []
    num_inputs = inputs.shape[1]
    num_outputs = outputs.shape[1]
    
    for di in range(len(inputs)):
        prob = pulp.LpProblem("DEA", pulp.LpMaximize)
        w_in = pulp.LpVariable.dicts("w_in", range(num_inputs), lowBound=0)
        w_out = pulp.LpVariable.dicts("w_out", range(num_outputs), lowBound=0)
        
        # Objective function
        prob += pulp.lpSum([w_out[o] * outputs[di][o] for o in range(num_outputs)]) - \
                pulp.lpSum([w_in[i] * inputs[di][i] for i in range(num_inputs)])
        
        # Add constraints to ensure no DMU's efficiency exceeds 1
        for i in range(len(inputs)):
            prob += pulp.lpSum([w_out[o] * outputs[i][o] for o in range(num_outputs)]) - \
                    pulp.lpSum([w_in[j] * inputs[i][j] for j in range(num_inputs)]) <= 1
        
        prob.solve()
        
        if prob.status == pulp.LpStatusOptimal:
            efficiencies.append(min(pulp.value(prob.objective), 1))
        else:
            efficiencies.append(None)
    
    # Display efficiencies
    for i, eff in enumerate(efficiencies):
        print(f"Efficiency of DMU {i + 1}: {eff}")
    
    # Optional: Add efficiencies to the dataframe and save to a new CSV
    df['efficiency'] = efficiencies
    
    if output_path:
        df.to_csv(output_path, index=False)
    
    return df


In [14]:
# Example usage:
data_path = 'data_phone_selection.csv'
input_features = ['baterry_mah', 'weight_oz', 'camera_mp', 'standby_time_hours']
output_feature = 'retail_price_usd'
#output_path = 'data_phone_selection_with_efficiencies.csv'

df_with_efficiencies = calculate_efficiencies(data_path, input_features, output_feature)
df_with_efficiencies

Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Dec 15 2019 

command line - /Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/pulp/solverdir/cbc/osx/64/cbc /var/folders/8j/3pr9zqy97g998nn_l0_dmlz80000gn/T/3330653529c04b98ad1109491ae2f172-pulp.mps -max -timeMode elapsed -branch -printingOptions all -solution /var/folders/8j/3pr9zqy97g998nn_l0_dmlz80000gn/T/3330653529c04b98ad1109491ae2f172-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 13 COLUMNS
At line 59 RHS
At line 68 BOUNDS
At line 69 ENDATA
Problem MODEL has 8 rows, 5 columns and 40 elements
Coin0008I MODEL read with 0 errors
Option for timeMode changed from cpu to elapsed
Presolve 8 (0) rows, 5 (0) columns and 40 (0) elements
0  Obj -0 Dual inf 1476.0082 (1)
0  Obj -0 Dual inf 1476.0082 (1)
3  Obj 1
Optimal - objective value 1
Optimal objective 1 - 3 iterations time 0.002
Option for printingOptions changed from normal to all
Total time (CPU seconds):  

,decision_making_unit,baterry_mah,weight_oz,camera_mp,standby_time_hours,retail_price_usd,efficiency
0,iPhone 5,1800,3.95,8.0,225.0,359.99,1.000000
1,HTC Windows 8,1800,4.60,8.0,300.0,399.99,1.000000
2,HTC Desire,1230,5.15,5.0,384.0,299.99,1.000000
3,Samsung Galaxy S4,2600,4.59,13.0,370.0,499.99,1.000000
4,Samsung Galaxy S4 Mini,1900,3.77,8.0,300.0,399.99,1.000000
5,Samsung Gusto 3,1000,3.50,1.3,770.4,149.99,1.000000
6,LG Cosmos,950,4.58,1.3,818.0,149.99,1.000000
7,Nokia Lumia 928,2000,5.75,8.7,606.0,399.99,0.941664


In [19]:
# Example usage:
data_path = 'data_phone_selection.csv'
input_features = ['baterry_mah', 'weight_oz', 'standby_time_hours']
output_feature = ['retail_price_usd',  'camera_mp']
#output_path = 'data_phone_selection_with_efficiencies.csv'

df_with_efficiencies = calculate_efficiencies(data_path, input_features, output_feature)
df_with_efficiencies

Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Dec 15 2019 

command line - /Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/pulp/solverdir/cbc/osx/64/cbc /var/folders/8j/3pr9zqy97g998nn_l0_dmlz80000gn/T/f8327ed6f71b4bf49e611e72c34c17ff-pulp.mps -max -timeMode elapsed -branch -printingOptions all -solution /var/folders/8j/3pr9zqy97g998nn_l0_dmlz80000gn/T/f8327ed6f71b4bf49e611e72c34c17ff-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 13 COLUMNS
At line 59 RHS
At line 68 BOUNDS
At line 69 ENDATA
Problem MODEL has 8 rows, 5 columns and 40 elements
Coin0008I MODEL read with 0 errors
Option for timeMode changed from cpu to elapsed
Presolve 8 (0) rows, 5 (0) columns and 40 (0) elements
0  Obj -0 Dual inf 3076.0082 (2)
0  Obj -0 Dual inf 3076.0082 (2)
3  Obj 1
Optimal - objective value 1
Optimal objective 1 - 3 iterations time 0.002
Option for printingOptions changed from normal to all
Total time (CPU seconds):  

,decision_making_unit,baterry_mah,weight_oz,camera_mp,standby_time_hours,retail_price_usd,efficiency
0,iPhone 5,1800,3.95,8.0,225.0,359.99,1.000000
1,HTC Windows 8,1800,4.60,8.0,300.0,399.99,1.000000
2,HTC Desire,1230,5.15,5.0,384.0,299.99,1.000000
3,Samsung Galaxy S4,2600,4.59,13.0,370.0,499.99,1.000000
4,Samsung Galaxy S4 Mini,1900,3.77,8.0,300.0,399.99,1.000000
5,Samsung Gusto 3,1000,3.50,1.3,770.4,149.99,0.299986
6,LG Cosmos,950,4.58,1.3,818.0,149.99,0.299986
7,Nokia Lumia 928,2000,5.75,8.7,606.0,399.99,0.857135
